### Strategy Types

# Key Questions for our taking strategy

# 1. Entry/Exit
# 2. Trade Sizing
# 3. Leverage

Import Libraries

In [ ]:
# Data and analysis libraries
import polars as pl                         # Fast dataframes for financial data
import numpy as np                          # Numerical computing library
from datetime import datetime, timedelta    # Date and time operations
import random


# Machine learning libraries
import torch                                # PyTorch framework
import torch.nn as nn                       # Neural network modules
import torch.optim as optim                 # Optimization algorithms
import research                             # Model building and training utilities


# Visualization and
import altair as alt                        # Interactive visualization library

# data sources
import binance                              # Binance market data utilities

Load Model

In [ ]:
import models
model = models.LinearModel(3)
# security alert
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()

Model Parameters

In [ ]:
research.print_model_params(model)

Strategy Development!

Load Time Series

In [ ]:
sym = 'BTCUSDT'
time_interval = '12h'

ts = pl.read_csv(f"{sym}_{time_interval}_ohlc.csv", try_parse_dates=True).sort('datetime')
ts

Add Target and Features

In [ ]:
sym = 'BTCUSDT'
forecast_horizon = 1
ts = research.add_log_return_features(ts, 'close', forecast_horizon, max_no_lags=3)
ts

Time Split

In [ ]:
test_size = 0.25
_, trades = research.timeseries_split(ts, test_size)
trades

Strategy Decision # 1: Entry / Exit Signal

 predicate example => we only want to trade if our y_hat is above or below a certain threshold
 time based => each row represents a roundtrip trade. trade to open position at start of interval, trade to close the position at end of interval. each row = 2 trades.

Add Model's Predictions

In [ ]:
target = 'close_log_return'
features = [f'{target}_lag_1',f'{target}_lag_2',f'{target}_lag_3']
trades = research.add_model_predictions(trades, model, features)
trades

Add Directional Signal

In [ ]:
trades = trades.with_columns(pl.col('y_hat').sign().alias('dir_signal'))
trades

Calculate Trade Log Return

In [ ]:
trades = trades.with_columns((pl.col('close_log_return')*pl.col('dir_signal')).alias('trade_log_return'))
trades

Calculate Cumulative Trade Log Return

In [ ]:
trades = trades.with_columns(pl.col('trade_log_return').cum_sum().alias('cum_trade_log_return'))
trades

Display Equity Curve (Log Space)

In [ ]:
research.plot_column(trades, 'cum_trade_log_return')

Key Strategy Decision #2: Trade Sizing

In [ ]:
capital = 100
ratio = 1.0 # you could experiment with different ratios like kelly criterion
trade_value = ratio * capital

# entry trade value and exit trade value
trades = trades.with_columns(
    pl.lit(trade_value).alias('entry_trade_value'),
    (trade_value * pl.col('trade_log_return').exp()).alias('exit_trade_value'),
    (trade_value / pl.col('open')).alias('trade_qty'),
).with_columns(
    (pl.col('trade_qty') * pl.col('dir_signal')).alias('signed_trade_qty'),
)

trades.select('datetime','open','close', 'trade_log_return','y_hat','entry_trade_value','exit_trade_value', 'signed_trade_qty')

Add Trade Gross PnL

In [ ]:
trades = trades.with_columns((pl.col('exit_trade_value') - pl.col('entry_trade_value')).alias('trade_gross_pnl'))
trades.select('datetime','open','close', 'trade_log_return','y_hat','entry_trade_value','exit_trade_value', 'signed_trade_qty',)

Add Transaction Fees

In [ ]:
taker_fee = binance.TAKER_FEE
maker_fee = binance.MAKER_FEE

trades = trades.with_columns(
    (pl.col('entry_trade_value') * taker_fee + pl.col('exit_trade_value') * taker_fee).alias('taker_fee'),
    (pl.col('entry_trade_value') * maker_fee + pl.col('exit_trade_value') * maker_fee).alias('maker_fee')
)

trades.select('datetime','open','close', 'trade_log_return','y_hat','entry_trade_value','exit_trade_value', 'signed_trade_qty', 'trade_gross_pnl','maker_fee','taker_fee')


Calculate Trade Net PnL

In [ ]:
trades = trades.with_columns(
    (pl.col('trade_gross_pnl')-pl.col('taker_fee')).alias('trade_net_taker_pnl'),
    (pl.col('trade_gross_pnl')-pl.col('maker_fee')).alias('trade_net_maker_pnl'),
)

trades.select('datetime','open','close', 'trade_log_return','y_hat','entry_trade_value','exit_trade_value', 'signed_trade_qty', 'trade_gross_pnl','trade_net_taker_pnl','trade_net_maker_pnl')

Display Equity Curves for Constant Sizing

In [ ]:
def equity_curve(capital, col_name, suffix):
    return (capital + (pl.col(col_name).cum_sum())).alias(f'equity_curve_{suffix}')

trades = trades.with_columns(
    equity_curve(capital, 'trade_net_taker_pnl', 'taker'),
    equity_curve(capital, 'trade_net_maker_pnl', 'maker'),
    equity_curve(capital, 'trade_gross_pnl', 'gross'),
)
trades.select('datetime','equity_curve_gross','equity_curve_taker','equity_curve_maker')


In [ ]:
research.plot_static_timeseries(trades, sym, 'equity_curve_taker', time_interval)

In [ ]:
research.plot_static_timeseries(trades, sym, 'equity_curve_maker', time_interval)

Calculate Total Net Return using Constant Sizing

In [ ]:
constant_sizing_net_return = trades['equity_curve_taker'][-1] / capital - 1
constant_sizing_net_return

Key Strategy Decision #3: Leverage

In [ ]:
leverage = 4
leverage * capital

In [ ]:
trades = research.add_compounding_trades(trades, capital, leverage, maker_fee, taker_fee)
trades

In [ ]:
trades['equity_curve_taker'][-1] / capital - 1